## Below is the code to re-create Figure 4

### Topic: Generating clusters with multiplicity and differential extinction

In [1]:
# Import necessary packages. 
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
import numpy as np
import pylab as py
import pdb
import os
import pylab as py

/Users/mwhosek/anaconda3/envs/astroconda2/lib/python3.6/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in /Users/mwhosek/models/models_2019_07/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [2]:
# First, we'll generate 2 cluster CMDs, one with multiplicity and one without. We'll 
# reuse the merged isochrone (iso_merged) with Cardelli+89 extinction law from Figure 2 
# for the purposes  of the plot (Age = 5 Myr, solar metallicity, A_Ks = 1.0 mag, dist=4000 pc).
# Both clusters will be 10^4 M_sun and a Kroupa+01 IMF

# Fetch isochrone
logAge = 6.70 # Age in log(years)
AKs = 1.0 # Ks filter extinction in mags
dist = 4000 # distance in parsecs
metallicity = 0 # metallicity in [M/H]
atm_func = atmospheres.get_merged_atmosphere
evo_merged = evolution.MISTv1()
redlaw = reddening.RedLawCardelli(3.1) # Rv = 3.1
filt_list = ['nirc2,J', 'nirc2,Kp']

iso_dir = 'iso_merged_r1/'
iso_merged = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_merged, atm_func=atm_func,
                                 filters=filt_list, red_law=redlaw,
                                 iso_dir=iso_dir, mass_sampling=3)

Changing to logg=4.00 for T= 31319 logg=3.99
Changing to logg=4.00 for T= 32012 logg=3.95
Changing to logg=4.00 for T= 32586 logg=3.90
Changing to logg=4.00 for T= 33107 logg=3.84
Changing to logg=4.00 for T= 33284 logg=3.81
Changing to logg=4.00 for T= 33336 logg=3.79
Changing to logg=4.00 for T= 33406 logg=3.78
Changing to logg=4.00 for T= 33350 logg=3.72
Changing to logg=4.00 for T= 33249 logg=3.68
Changing to logg=4.00 for T= 33013 logg=3.64
Changing to logg=4.00 for T= 32774 logg=3.61
Changing to logg=4.00 for T= 32504 logg=3.57
Changing to logg=4.00 for T= 31785 logg=3.50
Changing to logg=4.00 for T= 31100 logg=3.44
Changing to logg=3.50 for T= 29991 logg=3.33
Changing to logg=3.50 for T= 28921 logg=3.23
Changing to logg=3.50 for T= 28170 logg=3.17
Changing to logg=3.50 for T= 27430 logg=3.10
Changing to logg=3.50 for T= 26705 logg=3.04
Changing to logg=3.50 for T= 26477 logg=3.00
Changing to logg=3.50 for T= 27177 logg=3.04
Changing to logg=3.50 for T= 27919 logg=3.09
Changing t

In [3]:
# Now we can make the cluster. 
clust_mtot = 10**4.
clust_multiplicity = multiplicity.MultiplicityUnresolved()

# Multiplicity is defined in the IMF object
clust_imf_noMult = imf.Kroupa_2001(multiplicity=None)
clust_imf_Mult = imf.Kroupa_2001(multiplicity=clust_multiplicity)

In [4]:
# Make clusters
clust_noMult = synthetic.ResolvedCluster(iso_merged, clust_imf_noMult, clust_mtot)
clust_Mult = synthetic.ResolvedCluster(iso_merged, clust_imf_Mult, clust_mtot)

clust_noMult = clust_noMult.star_systems
clust_Mult = clust_Mult.star_systems

Found 8556 stars out of mass range
Found 6383 stars out of mass range
Found 1229 companions out of stellar mass range


/Users/mwhosek/anaconda3/envs/astroconda2/lib/python3.6/site-packages/astropy/table/column.py:991: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/Users/mwhosek/anaconda3/envs/astroconda2/lib/python3.6/site-packages/astropy/table/column.py:991: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)


In [5]:
# Next, we'll generate a cluster with multiplicity off *but* with 
# differential extinction. Note that we have to use the 
# ResolvedClusterDiffRedden object now, and we have to pass in dAKs and 
# the extinction law. 
#
# The amount of differential extinction in a system (i.e., difference between 
# the system extinction and the average extinction) is drawn from a Gaussian
# distribution centered at 0 and of width dAKs

dAKs = 0.05 # Width of differential extinction distribution in K-band, in mags
clust_noMult_diff = synthetic.ResolvedClusterDiffRedden(iso_merged, clust_imf_noMult, 
                                                        clust_mtot, dAKs)
clust_noMult_diff = clust_noMult_diff.star_systems

In [6]:
# Now we can make the final cluster figure
py.figure(figsize=(20,10))
py.subplots_adjust(left=0.08)
py.subplot(121)
py.plot(clust_Mult['m_nirc2_J'] - clust_Mult['m_nirc2_Kp'],
       clust_Mult['m_nirc2_J'], 'r.', label='With Multiples')
py.plot(clust_noMult['m_nirc2_J'] - clust_noMult['m_nirc2_Kp'],
       clust_noMult['m_nirc2_J'], 'k.', label='Without Multiples')
py.xlabel('NIRC2 J - Kp (mag)', fontsize=24)
py.ylabel('NIRC2 J (mag)', fontsize=24)
py.tick_params(axis='both', labelsize=20)
py.ylim(23, 8)
py.legend(fontsize=20)
py.subplot(122)
py.plot(clust_noMult_diff['m_nirc2_J'] - clust_noMult_diff['m_nirc2_Kp'],
       clust_noMult_diff['m_nirc2_J'], 'r.', label='With dAKs = 0.05 mag')
py.plot(clust_noMult['m_nirc2_J'] - clust_noMult['m_nirc2_Kp'],
       clust_noMult['m_nirc2_J'], 'k.', label='With dAKs = 0 mag')
py.xlabel('NIRC2 J - Kp (mag)', fontsize=24)
py.ylabel('NIRC2 J (mag)', fontsize=24)
py.ylim(23, 8)
py.tick_params(axis='both', labelsize=20)
py.legend(fontsize=20)
py.savefig('cluster_cmd.png')